Severe Climate Events: Disaster Declaration for states and counties

In [2]:
import pandas as pd

In [3]:
disasters_raw = pd.read_csv("disaster_declarations.csv")

In [4]:
disasters_raw.head()

,femaDeclarationString,disasterNumber,state,declarationType,declarationDate,fyDeclared,incidentType,declarationTitle,ihProgramDeclared,iaProgramDeclared,...,disasterCloseoutDate,fipsStateCode,fipsCountyCode,placeCode,designatedArea,declarationRequestNumber,lastIAFilingDate,hash,id,lastRefresh
0,FM-5444-TX,5444,TX,FM,2022-07-19T00:00:00.000Z,2022,Fire,CHALK MOUNTAIN FIRE,0,0,...,NaN,48,221,99221,Hood (County),22060,NaN,373c5ec27998afc08a53302dae796f476b1a6546,867be42a-71d5-4f13-aa21-d91e0a6fd577,2022-07-20T21:21:23.941Z
1,FM-5436-NE,5436,NE,FM,2022-04-23T00:00:00.000Z,2022,Fire,ROAD 702 FIRE,0,0,...,NaN,31,63,99063,Frontier (County),22034,NaN,ea3487ef36cff455236ce4c63d32fb8b5412bcef,e671348b-9782-42df-99f4-d38b8b1a89e6,2022-07-20T21:21:23.942Z
2,FM-5444-TX,5444,TX,FM,2022-07-19T00:00:00.000Z,2022,Fire,CHALK MOUNTAIN FIRE,0,0,...,NaN,48,425,99425,Somervell (County),22060,NaN,1f35dd8137e1b4cf003fb73d53d8aaaf642e6190,40f3ff75-0b80-4d25-8e85-156cd6a6f40b,2022-07-20T21:21:23.943Z
3,FM-5436-NE,5436,NE,FM,2022-04-23T00:00:00.000Z,2022,Fire,ROAD 702 FIRE,0,0,...,NaN,31,65,99065,Furnas (County),22034,NaN,9d813a845ab86f546bdf642fda53d3d4a0fbd098,de47838f-32db-4058-a34c-64997333789e,2022-07-20T21:21:23.943Z
4,FM-5436-NE,5436,NE,FM,2022-04-23T00:00:00.000Z,2022,Fire,ROAD 702 FIRE,0,0,...,NaN,31,145,99145,Red Willow (County),22034,NaN,d436856ce1d5205fa78597b93661fb27d3cea796,483c03b7-160a-414e-bb4f-5ccecc18d94a,2022-07-20T21:21:23.944Z


In [5]:
disasters_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63989 entries, 0 to 63988
Data columns (total 24 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   femaDeclarationString     63989 non-null  object
 1   disasterNumber            63989 non-null  int64 
 2   state                     63989 non-null  object
 3   declarationType           63989 non-null  object
 4   declarationDate           63989 non-null  object
 5   fyDeclared                63989 non-null  int64 
 6   incidentType              63989 non-null  object
 7   declarationTitle          63989 non-null  object
 8   ihProgramDeclared         63989 non-null  int64 
 9   iaProgramDeclared         63989 non-null  int64 
 10  paProgramDeclared         63989 non-null  int64 
 11  hmProgramDeclared         63989 non-null  int64 
 12  incidentBeginDate         63989 non-null  object
 13  incidentEndDate           55609 non-null  object
 14  disasterCloseoutDate  

In [6]:
disasters = disasters_raw[['id','disasterNumber', 'state','fipsStateCode','fipsCountyCode','placeCode','declarationDate','fyDeclared','incidentType',
    'ihProgramDeclared','iaProgramDeclared','paProgramDeclared','hmProgramDeclared',
    'incidentBeginDate','incidentEndDate','designatedArea']]

Calculating the number of disasters by incident type and removing the ones that are not climate disasters. 

In [71]:
#disasters.incidentType.value_counts()
#disasters.loc[disasters["incidentType"] == "Severe Storm(s)"]
#disasters.loc[disasters["disasterNumber"] == 4672]

state_wide_disasters = disasters.loc[disasters["designatedArea"] == "Statewide"]
#state_wide_disasters
#state_wide_disasters_no_dupl = state_wide_disasters.drop_duplicates(subset='disasterNumber', keep="first")
#state_wide_disasters_no_dupl

,id,disasterNumber,state,fipsStateCode,fipsCountyCode,placeCode,declarationDate,fyDeclared,incidentType,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared,incidentBeginDate,incidentEndDate,designatedArea
409,0a47cc92-3705-4331-8c1a-da55a165c7fd,5133,CO,8,0,0,2016-07-10T00:00:00.000Z,2016,Fire,0,0,1,0,2016-07-09T00:00:00.000Z,2016-07-14T00:00:00.000Z,Statewide
8725,bdfa7fae-eb8d-4346-8025-f1b219c5eb50,4027,RI,44,0,0,2011-09-03T00:00:00.000Z,2011,Hurricane,0,0,0,1,2011-08-27T00:00:00.000Z,2011-08-29T00:00:00.000Z,Statewide
12662,e15d091a-0c63-4068-803e-53dd0ff97c6b,3459,LA,22,0,0,2020-03-13T00:00:00.000Z,2020,Biological,0,0,1,0,2020-01-20T00:00:00.000Z,NaN,Statewide
16766,a96786fe-4a6c-47b8-b88c-d1bdb595bf77,3356,PA,42,0,0,2012-10-29T00:00:00.000Z,2013,Hurricane,0,0,1,0,2012-10-26T00:00:00.000Z,2012-11-08T00:00:00.000Z,Statewide
19398,f785e0d8-789e-4e0c-9814-42dce97e7c18,3281,MO,29,0,0,2007-12-12T00:00:00.000Z,2008,Severe Ice Storm,0,0,1,0,2007-12-08T00:00:00.000Z,2007-12-15T00:00:00.000Z,Statewide
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63896,f0d98626-e256-4cdf-b264-510b0e51570d,182,MT,30,0,0,1964-12-18T00:00:00.000Z,1965,Snowstorm,0,1,1,1,1964-12-18T00:00:00.000Z,1964-12-18T00:00:00.000Z,Statewide
63897,68a9f1c4-37f2-44cd-a97a-0e52dbe5acf2,104,OK,40,0,0,1960-07-15T00:00:00.000Z,1960,Severe Storm,0,1,1,1,1960-07-15T00:00:00.000Z,1960-07-15T00:00:00.000Z,Statewide
63898,4b63b1ea-09ee-40a1-bda9-2bc3326e8054,58,PA,42,0,0,1956-05-21T00:00:00.000Z,1956,Severe Storm,0,1,1,1,1956-05-21T00:00:00.000Z,1956-05-21T00:00:00.000Z,Statewide
63942,6ce9a6f2-7a9a-4ac4-bf4a-86cc91c23590,56,NC,37,0,0,1956-04-24T00:00:00.000Z,1956,Severe Storm,0,1,1,1,1956-04-24T00:00:00.000Z,1956-04-24T00:00:00.000Z,Statewide


In [33]:
disasters.incidentType.value_counts()
disaster_type_list = disasters.incidentType.unique().tolist()
disaster_type_list
#removing severe storm(s) since they are only 4 which are statewide emergencies that are already covered by severe storm- duplicates
unwanted_disasters = ['Biological','Toxic Substances','Chemical','Terrorist','Human Cause','Fishing Losses', 'Severe Storm(s)']
climate_disasters_list = [ele for ele in disaster_type_list if ele not in unwanted_disasters]
climate_disasters = disasters.loc[disasters['incidentType'].isin(climate_disasters_list)]
climate_disasters

In [34]:
climate_disasters.dtypes
climate_disasters["declarationDate"] = pd.to_datetime(climate_disasters["declarationDate"])
climate_disasters['Year'] = climate_disasters['declarationDate'].dt.strftime('%Y')

/tmp/user/23091/ipykernel_1960573/2505089607.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  climate_disasters["declarationDate"] = pd.to_datetime(climate_disasters["declarationDate"])
/tmp/user/23091/ipykernel_1960573/2505089607.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  climate_disasters['Year'] = climate_disasters['declarationDate'].dt.strftime('%Y')


In [35]:
climate_disasters.head()

,id,disasterNumber,state,fipsStateCode,fipsCountyCode,placeCode,declarationDate,fyDeclared,incidentType,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared,incidentBeginDate,incidentEndDate,designatedArea,Year
36,036f0c80-8f7d-4551-a29b-92a7923f93a6,1606,TX,48,97,99097,2005-09-24 00:00:00+00:00,2005,Hurricane,0,0,1,1,2005-09-23T00:00:00.000Z,2005-10-14T00:00:00.000Z,Cooke (County),2005
37,d816361f-2c0e-431a-90e5-a836be2ffaf6,1606,TX,48,101,99101,2005-09-24 00:00:00+00:00,2005,Hurricane,0,0,1,1,2005-09-23T00:00:00.000Z,2005-10-14T00:00:00.000Z,Cottle (County),2005
57,a3c1ab84-73dd-4e1b-988f-9968c3a6f005,1971,AL,1,95,99095,2011-04-28 00:00:00+00:00,2011,Severe Storm,1,0,1,1,2011-04-15T00:00:00.000Z,2011-05-31T00:00:00.000Z,Marshall (County),2011
114,00c02cd1-ec76-47d1-a5e7-964c4fb6c7b5,4627,DE,10,3,99003,2021-10-24 00:00:00+00:00,2022,Hurricane,0,0,1,1,2021-09-01T00:00:00.000Z,2021-09-07T00:00:00.000Z,New Castle (County),2021
117,3fd860db-e27c-4211-9a69-b314bc68d306,1971,AL,1,119,99119,2011-04-28 00:00:00+00:00,2011,Severe Storm,1,0,1,1,2011-04-15T00:00:00.000Z,2011-05-31T00:00:00.000Z,Sumter (County),2011


In [36]:
climate_last_23 = climate_disasters.loc[(climate_disasters['Year'] >= '2000')
                     & (climate_disasters['Year'] <= '2023')]
climate_last_23

,id,disasterNumber,state,fipsStateCode,fipsCountyCode,placeCode,declarationDate,fyDeclared,incidentType,ihProgramDeclared,iaProgramDeclared,paProgramDeclared,hmProgramDeclared,incidentBeginDate,incidentEndDate,designatedArea,Year
36,036f0c80-8f7d-4551-a29b-92a7923f93a6,1606,TX,48,97,99097,2005-09-24 00:00:00+00:00,2005,Hurricane,0,0,1,1,2005-09-23T00:00:00.000Z,2005-10-14T00:00:00.000Z,Cooke (County),2005
37,d816361f-2c0e-431a-90e5-a836be2ffaf6,1606,TX,48,101,99101,2005-09-24 00:00:00+00:00,2005,Hurricane,0,0,1,1,2005-09-23T00:00:00.000Z,2005-10-14T00:00:00.000Z,Cottle (County),2005
57,a3c1ab84-73dd-4e1b-988f-9968c3a6f005,1971,AL,1,95,99095,2011-04-28 00:00:00+00:00,2011,Severe Storm,1,0,1,1,2011-04-15T00:00:00.000Z,2011-05-31T00:00:00.000Z,Marshall (County),2011
114,00c02cd1-ec76-47d1-a5e7-964c4fb6c7b5,4627,DE,10,3,99003,2021-10-24 00:00:00+00:00,2022,Hurricane,0,0,1,1,2021-09-01T00:00:00.000Z,2021-09-07T00:00:00.000Z,New Castle (County),2021
117,3fd860db-e27c-4211-9a69-b314bc68d306,1971,AL,1,119,99119,2011-04-28 00:00:00+00:00,2011,Severe Storm,1,0,1,1,2011-04-15T00:00:00.000Z,2011-05-31T00:00:00.000Z,Sumter (County),2011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58987,e5a603dd-b504-45f9-b30b-cfe7694f1055,1311,GA,13,223,99223,2000-01-28 00:00:00+00:00,2000,Severe Storm,0,0,1,1,2000-01-22T00:00:00.000Z,2000-02-01T00:00:00.000Z,Paulding (County),2000
59018,f607f5c6-1f6f-4ff7-a7f0-bda45598c22a,1311,GA,13,187,99187,2000-01-28 00:00:00+00:00,2000,Severe Storm,0,0,1,1,2000-01-22T00:00:00.000Z,2000-02-01T00:00:00.000Z,Lumpkin (County),2000
59019,5a26214d-7749-4d1f-968d-c7f00cd6b72e,1311,GA,13,311,99311,2000-01-28 00:00:00+00:00,2000,Severe Storm,0,0,1,1,2000-01-22T00:00:00.000Z,2000-02-01T00:00:00.000Z,White (County),2000
59053,a01f3466-8390-46ed-bc53-5304f1d9f54a,1311,GA,13,231,99231,2000-01-28 00:00:00+00:00,2000,Severe Storm,0,0,1,1,2000-01-22T00:00:00.000Z,2000-02-01T00:00:00.000Z,Pike (County),2000


If we take all years- we have 52,445 climate-related disasters. If we take from 2000 onwards (2000-2023 included), we get 33,629 climate related disasters. 

In [37]:
climate_last_23.state.value_counts()
#climate_last_23.designatedArea.value_counts()

TX    2363
MO    1845
LA    1832
OK    1603
KY    1509
FL    1404
GA    1272
VA    1227
NC    1219
KS    1175
MS    1134
PR    1072
AR     971
AL     971
IA     953
NE     921
NY     895
TN     790
IN     781
WV     713
SC     711
OH     680
ND     675
MN     667
SD     634
PA     600
IL     574
WI     384
CA     372
WA     351
NJ     343
MD     253
MA     240
MI     220
MT     214
ME     212
VT     197
OR     194
NH     193
CO     183
CT     171
NM     150
AK     125
ID     107
AZ      87
NV      75
UT      72
RI      65
HI      45
MP      38
VI      35
WY      32
AS      31
DE      30
DC      17
GU       8
FM       8
Name: state, dtype: int64

For political lens: get the name of the states that have been affected- maybe the highest ones?

For financial lens: counties that are yes and what all was provided to them? 

For socio-economic: counties that were affected- get the demographic breakdown, what are the relationships between the most heavily affected ones and their socio-economic breakdown?

In [38]:
#Renaming climate columns
climate_last_23.columns = ['record_id', 'disaster_number', 'state', 'state_code',
                          'county_code','place_code','dec_date', 'fiscal_year',
                        'disaster_type','ih_program','ia_program', 'pa_program',
                        'hm_progam','begin_date', 'end_date', 'area_name','year']
cliate_last_23 = climate_last_23.drop(columns=['fiscal_year'])
duplicateRows = climate_last_23[climate_last_23.duplicated()]
len(duplicateRows)
duplicateRows = climate_last_23[climate_last_23.duplicated(['record_id'])]
len(duplicateRows)

0

Does not look like there are any duplicates that we need to worry about. 
Hence, dropping the record_id column.

In [39]:
climate_last_23 = climate_last_23.drop(columns=['record_id'])

In [40]:
climate_last_23.hm_progam.value_counts()/33629

1    0.628921
0    0.371495
Name: hm_progam, dtype: float64

In [41]:
climate_last_23.ih_program.value_counts()/33629

0    0.807577
1    0.192840
Name: ih_program, dtype: float64

In [42]:
climate_last_23.ia_program.value_counts()/33629

0    0.860656
1    0.139760
Name: ia_program, dtype: float64

In [43]:
climate_last_23.pa_program.value_counts()/33629


1    0.941925
0    0.058491
Name: pa_program, dtype: float64

Hazard Mitigation declared: 21,136 disasters (62.85%)

Individual and Households: 27,144 disasters (80.71%)

Individual Assistance: 28,929 disasters (86.02%)

Public Assistance: 31,662 disasters. (94.15%)

Keeping only Public assistance column as decided by looking at other dataset.



In [44]:
climate_last_23 = climate_last_23.drop(columns=['ih_program','ia_program','hm_progam'])

Questions to answer:

1. What are the type of disasters that are occuring?

2. What % of disasters are getting some kind of economic assistance?

- Where does that economic assistance acctually flow to- which states?

3. Which months are causing what kind of disasters? Which month has the most disasters?

4. Which counties face the most disasters? 

5. Can add the public assistance data- map it to the specific disasters-
based on disaster numbers. See how much money goes to each state. Which kind of disaster gets the most amount of money as well. 
Use public assistance cause it is the largest grant that FEMA provides. 



Adding final_county_code by cleaning state code and county_code and merging them
together. 

In [45]:
climate_last_23["state_code"] = climate_last_23.state_code.map("{:02d}".format)

In [46]:
climate_last_23["county_code"] = climate_last_23.county_code.map("{:03d}".format)

In [47]:
climate_last_23['full_county_code'] = climate_last_23['state_code'] + climate_last_23['county_code']
climate_last_23
#climate_last_23 = climate_last_23.drop(columns=['place_code'])
#climate_last_23

,disaster_number,state,state_code,county_code,place_code,dec_date,fiscal_year,disaster_type,pa_program,begin_date,end_date,area_name,year,full_county_code
36,1606,TX,48,097,99097,2005-09-24 00:00:00+00:00,2005,Hurricane,1,2005-09-23T00:00:00.000Z,2005-10-14T00:00:00.000Z,Cooke (County),2005,48097
37,1606,TX,48,101,99101,2005-09-24 00:00:00+00:00,2005,Hurricane,1,2005-09-23T00:00:00.000Z,2005-10-14T00:00:00.000Z,Cottle (County),2005,48101
57,1971,AL,01,095,99095,2011-04-28 00:00:00+00:00,2011,Severe Storm,1,2011-04-15T00:00:00.000Z,2011-05-31T00:00:00.000Z,Marshall (County),2011,01095
114,4627,DE,10,003,99003,2021-10-24 00:00:00+00:00,2022,Hurricane,1,2021-09-01T00:00:00.000Z,2021-09-07T00:00:00.000Z,New Castle (County),2021,10003
117,1971,AL,01,119,99119,2011-04-28 00:00:00+00:00,2011,Severe Storm,1,2011-04-15T00:00:00.000Z,2011-05-31T00:00:00.000Z,Sumter (County),2011,01119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58987,1311,GA,13,223,99223,2000-01-28 00:00:00+00:00,2000,Severe Storm,1,2000-01-22T00:00:00.000Z,2000-02-01T00:00:00.000Z,Paulding (County),2000,13223
59018,1311,GA,13,187,99187,2000-01-28 00:00:00+00:00,2000,Severe Storm,1,2000-01-22T00:00:00.000Z,2000-02-01T00:00:00.000Z,Lumpkin (County),2000,13187
59019,1311,GA,13,311,99311,2000-01-28 00:00:00+00:00,2000,Severe Storm,1,2000-01-22T00:00:00.000Z,2000-02-01T00:00:00.000Z,White (County),2000,13311
59053,1311,GA,13,231,99231,2000-01-28 00:00:00+00:00,2000,Severe Storm,1,2000-01-22T00:00:00.000Z,2000-02-01T00:00:00.000Z,Pike (County),2000,13231


What are the type of disasters and frequency of disasters? (Grouping by disasters and plotting them over time)

In [48]:
#How many disasters have we had?
climate_last_23.disaster_number.value_counts()
# Have had 1584 disasters 
#(climate_last_23['disaster_number'].value_counts() > 10)
#climate_last_23[["disaster_number"]].describe()
#climate_last_23['disaster_number_count'] = climate_last_23['disaster_number'].value_counts()


3261    255
1606    254
4586    254
3216    254
3554    254
       ... 
1446      1
4205      1
4202      1
1430      1
3581      1
Name: disaster_number, Length: 1585, dtype: int64

In [49]:
#climate_last_23.set_index(["disaster_type"], inplace = True, drop= False)
cl= climate_last_23.disaster_number.value_counts().to_dict()
climate_last_23['number_of_counties_affected'] = climate_last_23['disaster_number'].map(cl)

In [50]:
#climate_last_23.loc[1606]
climate_last_23['end_date']= pd.to_datetime(climate_last_23['end_date'])
climate_last_23['begin_date']= pd.to_datetime(climate_last_23['begin_date'])


climate_last_23['length_of_disaster'] = (climate_last_23['end_date'] - climate_last_23['begin_date']).dt.days
climate_last_23
#climate_last_23.to_csv('climate_disasters_2000_2023_clean.csv')

,disaster_number,state,state_code,county_code,place_code,dec_date,fiscal_year,disaster_type,pa_program,begin_date,end_date,area_name,year,full_county_code,number_of_counties_affected,length_of_disaster
36,1606,TX,48,097,99097,2005-09-24 00:00:00+00:00,2005,Hurricane,1,2005-09-23 00:00:00+00:00,2005-10-14 00:00:00+00:00,Cooke (County),2005,48097,254,21.0
37,1606,TX,48,101,99101,2005-09-24 00:00:00+00:00,2005,Hurricane,1,2005-09-23 00:00:00+00:00,2005-10-14 00:00:00+00:00,Cottle (County),2005,48101,254,21.0
57,1971,AL,01,095,99095,2011-04-28 00:00:00+00:00,2011,Severe Storm,1,2011-04-15 00:00:00+00:00,2011-05-31 00:00:00+00:00,Marshall (County),2011,01095,67,46.0
114,4627,DE,10,003,99003,2021-10-24 00:00:00+00:00,2022,Hurricane,1,2021-09-01 00:00:00+00:00,2021-09-07 00:00:00+00:00,New Castle (County),2021,10003,1,6.0
117,1971,AL,01,119,99119,2011-04-28 00:00:00+00:00,2011,Severe Storm,1,2011-04-15 00:00:00+00:00,2011-05-31 00:00:00+00:00,Sumter (County),2011,01119,67,46.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58987,1311,GA,13,223,99223,2000-01-28 00:00:00+00:00,2000,Severe Storm,1,2000-01-22 00:00:00+00:00,2000-02-01 00:00:00+00:00,Paulding (County),2000,13223,51,10.0
59018,1311,GA,13,187,99187,2000-01-28 00:00:00+00:00,2000,Severe Storm,1,2000-01-22 00:00:00+00:00,2000-02-01 00:00:00+00:00,Lumpkin (County),2000,13187,51,10.0
59019,1311,GA,13,311,99311,2000-01-28 00:00:00+00:00,2000,Severe Storm,1,2000-01-22 00:00:00+00:00,2000-02-01 00:00:00+00:00,White (County),2000,13311,51,10.0
59053,1311,GA,13,231,99231,2000-01-28 00:00:00+00:00,2000,Severe Storm,1,2000-01-22 00:00:00+00:00,2000-02-01 00:00:00+00:00,Pike (County),2000,13231,51,10.0


Which disasters go on for the longest? 

In [51]:
#climate_by_dis_num = climate_last_23.groupby(['disaster_type', 'disaster_number'])
#climate_by_dis_num.first()

Total number of disasters each year?

In [52]:
#climate_last_23.groupby('disaster_number')['state_code'].unique().count()

In [53]:
#climate_last_23.loc[climate_last_23["disaster_number"]==1311]

In [54]:
#climate_last_23.full_county_code.value_counts()
climate_dict = {k: f.groupby('disaster_number')['year'].unique().apply(list).to_dict()
     for k, f in climate_last_23.groupby('disaster_type')}

In [55]:
#only keep one county row per disaster- since we just need to find the total number of events
climate_last_23_summary = climate_last_23.drop(columns=['dec_date','fiscal_year','pa_program','begin_date', 'end_date', 'area_name','full_county_code', 'length_of_disaster' ])
climate_last_23_summary = climate_last_23_summary.drop_duplicates(subset='disaster_number', keep="first")
climate_last_23_summary = climate_last_23_summary.drop(columns=['disaster_number','state_code','county_code','number_of_counties_affected'])
#climate_last_23_summary= climate_last_23_summary.drop('disaster_number',axis=1)
#climate_last_23_summary
climate_last_23_summary= climate_last_23_summary[['year','state','disaster_type']]
climate_last_23_summary
climate_last_23_summary.duplicated()
climate_last_23_summary= climate_last_23_summary.groupby(climate_last_23_summary.columns.tolist(),as_index=False).size()
climate_last_23_summary.rename(columns = {"size": "total_number_of_events"}, inplace = True)
climate_last_23_summary


,year,state,disaster_type,total_number_of_events
0,2000,AK,Severe Storm,1
1,2000,AL,Severe Storm,3
2,2000,AR,Severe Ice Storm,2
3,2000,AZ,Severe Storm,1
4,2000,CA,Earthquake,1
...,...,...,...,...
1101,2023,AL,Severe Storm,1
1102,2023,CA,Flood,2
1103,2023,GA,Severe Storm,1
1104,2023,ND,Snowstorm,1
